In [ ]:
# import necessary modules
from datetime import datetime
import pandas as pd
import db_connection
conn = db_connection.db_conn

In [ ]:
# print start of the job
start_time = datetime.now()
print('Data Load job started at at {}'.format(start_time))

In [ ]:
# import data from the reports folder (staging environment)
df_raw = pd.read_csv('Reports/Combined_pcard_expenditures.csv')
print(df_raw.shape)
df_raw.head()

In [ ]:
# # load the raw data into the PostgreSQL database
table_name = 'raw_data_transactions'
df_raw.to_sql(table_name, conn, if_exists='replace', index=False)